In [1]:
!pip install -q sentence-transformers langchain pinecone-client


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.5 MB/s eta 0:00:00


In [2]:
!pip install -q requests beautifulsoup4


In [3]:
!pip install python-dotenv

In [4]:
!pip install -U langchain-community # install langchain-community which contains ChromaDB

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [5]:
import os
import json

def save_to_json(data, output_directory):
    # Ensure the output directory exists
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Define the file path
    output_file = os.path.join(output_directory, "scraped_data.json")

    # Save data to JSON file
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

    print(f"Data saved to {output_file}")


In [6]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 80.2 MB/s eta 0:00:00
 

In [7]:
!pip install langchain-huggingface


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 31.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.5.23 requires tokenizers<=0.20.3,>=0.13.2, but you have tokenizers 0.21.0 which is incompatible.


In [9]:
import torch
torch.cuda.empty_cache()

In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import json
from dotenv import load_dotenv, find_dotenv
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_huggingface import HuggingFacePipeline
from transformers import pipeline
import torch
from sentence_transformers import SentenceTransformer


# Load environment variables
load_dotenv(find_dotenv(), override=True)

# Function to scrape data from a URL
def scrapeData(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"
    }

    response = requests.get(url, headers=headers)

    print(f"Fetching {url} - Status Code: {response.status_code}")

    if response.status_code != 200:
        return {
            "url": url,
            "error": f"Failed to fetch content, status code: {response.status_code}"
        }

    soup = BeautifulSoup(response.text, "html.parser")

    structured_data = {
        "url": url,
        "headings": [],
        "paragraphs": [],
    }

    # Extract headings
    for heading in range(1, 7):
        for tag in soup.find_all(f"h{heading}"):
            text = tag.get_text(strip=True)
            if text:
                structured_data["headings"].append(text)

    # Extract paragraphs
    for p in soup.find_all("p"):
        text = p.get_text(strip=True)
        if text:
            structured_data["paragraphs"].append(text)

    return structured_data

# Function to save scraped data to a JSON file
def save_to_json(data, directory):
    os.makedirs(directory, exist_ok=True)
    file_path = os.path.join(directory, 'scraped_data.json')
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=4)
    print(f"Data saved to {file_path}")

# Function to chunk text data
def chunk_data(data, chunk_size=256):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0)

    combined_text = []
    if 'headings' in data:
        combined_text.extend(data['headings'])
    if 'paragraphs' in data:
        combined_text.extend(data['paragraphs'])

    full_text = "\n\n".join(combined_text)
    chunks = text_splitter.split_text(full_text)

    document_chunks = [Document(page_content=chunk) for chunk in chunks]

    return document_chunks

# Function to create embeddings and store them in Chroma
def create_embeddings_chroma(chunks, persist_directory='./chroma_db'):
    # Use CPU for SentenceTransformer
    device = 'cpu'

    # Create embedding model
    embedding = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')

    # Create Chroma vector store
    vector_store = Chroma.from_documents(
        documents=chunks,
        embedding=embedding,
        persist_directory=persist_directory
    )

    return vector_store

# Function to set up GPT-Neo or similar text generation model
def setup_text_generator():
    device = 0 if torch.cuda.is_available() else -1
    generator = pipeline('text-generation', model='EleutherAI/gpt-neo-2.7B', max_length=512, device=device)
    return HuggingFacePipeline(pipeline=generator)

# Function to handle question-answering using ConversationalRetrievalChain
def ask_question(q, chain):
    try:
        result = chain.invoke({'question': q})
        return result.get('answer', "I'm not sure about that.")
    except Exception as e:
        return f"Error: {e}"

# Main function
def main():
    url = "https://www.uchicago.edu/"
    output_directory = "./scraped_data"

    # Scrape data
    scraped_data = scrapeData(url)
    if "error" in scraped_data:
        print(scraped_data["error"])
        return

    save_to_json(scraped_data, output_directory)

    # Chunk data
    chunks = chunk_data(scraped_data)

    # Create vector store
    vector_store = create_embeddings_chroma(chunks)

    # Setup text generator
    llm = setup_text_generator()

    # Configure retriever and memory
    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 5})
    memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

    # Build conversational retrieval chain
    crc = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory,
        chain_type='stuff',
        verbose=False
    )

    # Interactive chat loop
    while True:
        q = input('Your question: ')
        if q.lower() in ['exit', 'quit', 'bye']:
            print('Bye bye!')
            break
        result = ask_question(q, crc)
        print(result)
        print('-' * 100)

if __name__ == "__main__":
    main()


Fetching https://www.uchicago.edu/ - Status Code: 200
Data saved to ./scraped_data/scraped_data.json


Device set to use cpu


Your question: What are the key take aways


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
